In [78]:
from bs4 import BeautifulSoup
import requests
import regex as re
import pandas as pd
import os
import json
import random
import yfinance as yf

In [110]:
class Yahoo:
    def __init__(self, url : str = ''):
        self.url = url
        self.headers_list = [
                        'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36',
                        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
                        "Mozilla/5.0 (Windows NT 10.0; rv:91.0) Gecko/20100101 Firefox/91.0",
                        "Mozilla/5.0 (Windows NT 10.0; rv:78.0) Gecko/20100101 Firefox/78.0",
                        "Mozilla/5.0 (X11; Linux x86_64; rv:95.0) Gecko/20100101 Firefox/95.0",
                        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15',
                        'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0',
                        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
                        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0',
                        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'
                        ] 
        self.soup = ""
        self.headers = {'User-Agent' : random.choice(self.headers_list) }

    def get_soup(self):
        r = requests.get(self.url, headers = self.headers, timeout=1000)
        self.soup = BeautifulSoup(r.content, "lxml")
        
        return self.soup
    
    def get_soupTextYahoo(self):
        if not self.soup:
            soup = self.get_soup()
        
        soupText = soup.find_all("div", {"class":"caas-body"})[0]
        
        return str(soupText.text)

class NBP(Yahoo):
    def __init__(self, url : str = ''):
        super().__init__(url) 
        self.soup = ""
        self.offertListLinks = []
        
    def make_offertListLinks(self):
        if not self.soup:
            soup = self.get_soup()
        
        classSoup = soup.find_all("div", {"class":"product-card"})
        
        hrefs = []
        for item in classSoup[:-1]:
            revenue_symbol = re.compile('(?<=href=")(.*?)(?=">)')
            link = re.findall(revenue_symbol, str(item))[0]
            link = f"https://www.obligacjeskarbowe.pl/{link}"
            self.offertListLinks.append(link)
        
        return self.offertListLinks
    


In [111]:
def show_news(stock_name= "MCD", time=None):
    news_data = yf.Ticker(stock_name).news
    filtered_news = [article for article in news_data if stock_name in article['relatedTickers']]


    news_info = []
    for news in filtered_news[-3:]:
        title = news['title']
        link = news['link']
        scrap = Yahoo(link)
        article = scrap.get_soupTextYahoo()
        news_info.append((title, link, article))

    return json.dumps(news_info), None



In [112]:
scrap = Yahoo("https://finance.yahoo.com/news/yum-brands-yum-report-q3-155700403.html")

nbp = NBP("https://www.obligacjeskarbowe.pl/")
test = nbp.make_offertListLinks()

In [113]:
test

['https://www.obligacjeskarbowe.pl//oferta-obligacji/obligacje-3-miesieczne-ots/ots0124/',
 'https://www.obligacjeskarbowe.pl//oferta-obligacji/obligacje-roczne-ror/ror1024/',
 'https://www.obligacjeskarbowe.pl//oferta-obligacji/obligacje-2-letnie-dor/dor1025/',
 'https://www.obligacjeskarbowe.pl//oferta-obligacji/obligacje-3-letnie-tos/tos1026/',
 'https://www.obligacjeskarbowe.pl//oferta-obligacji/obligacje-4-letnie-coi/coi1027/',
 'https://www.obligacjeskarbowe.pl//oferta-obligacji/obligacje-10-letnie-edo/edo1033/',
 'https://www.obligacjeskarbowe.pl//oferta-obligacji/obligacje-6-letnie-ros/ros1029/',
 'https://www.obligacjeskarbowe.pl//oferta-obligacji/obligacje-12-letnie-rod/rod1035/']